<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/07-RAG_Improve_Chunking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables

In [1]:
!pip install -q llama-index==0.9.21 openai==1.6.0 tiktoken==0.5.2 chromadb==0.4.21 kaleido==0.2.1 python-multipart==0.0.6 cohere==4.39

In [2]:
import os

# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["OPENAI_API_KEY"] = ""

In [3]:
import nest_asyncio

nest_asyncio.apply()

# Load a Model

In [4]:
from llama_index.llms import OpenAI

llm = OpenAI(temperature=0.9, model="gpt-3.5-turbo", max_tokens=512)

# Create a VectoreStore

In [5]:
import chromadb

# create client and a new collection
# chromadb.EphemeralClient saves data in-memory.
chroma_client = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = chroma_client.create_collection("mini-llama-articles")

In [6]:
from llama_index.vector_stores import ChromaVectorStore

# Define a storage context object using the created vector database.
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Load the Dataset (CSV)

## Download

The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model. Read the dataset as a long string.

In [7]:
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv
# !curl -o ./mini-llama-articles.csv https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  169k  100  169k    0     0   868k      0 --:--:-- --:--:-- --:--:--  869k


## Read File

In [8]:
import csv

rows = []

# Load the file as a JSON
with open("./mini-llama-articles.csv", mode="r", encoding="utf-8") as file:
  csv_reader = csv.reader(file)

  for idx, row in enumerate( csv_reader ):
    if idx == 0: continue; # Skip header row
    rows.append( row )

# The number of characters in the dataset.
len( rows )

14

# Convert to Document obj

In [9]:
from llama_index import Document

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [Document(text=row[1], metadata={"title": row[0], "url": row[2], "source_name": row[3]}) for row in rows]

# By default, the node/chunks ids are set to random uuids. To ensure same id's per run, we manually set them.
for idx, doc in enumerate(documents):
    doc.id_ = f"doc_{idx}"

# Transforming

In [10]:
from llama_index.text_splitter import TokenTextSplitter
from llama_index.schema import BaseNode
import hashlib

def deterministic_id_func(i: int, doc: BaseNode) -> str:
    """Deterministic ID function for the text splitter.
    This will be used to generate a unique repeatable identifier for each node."""
    unique_identifier = doc.id_ + str(i)
    hasher = hashlib.sha256()
    hasher.update(unique_identifier.encode('utf-8')) 
    return hasher.hexdigest()

text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128, id_func=deterministic_id_func)

In [11]:
from llama_index.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
)
from llama_index.embeddings import OpenAIEmbedding
from llama_index.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        QuestionsAnsweredExtractor(questions=3, llm=llm),
        SummaryExtractor(summaries=["prev", "self"], llm=llm),
        KeywordExtractor(keywords=10, llm=llm),
        OpenAIEmbedding(),
    ],
    vector_store=vector_store
)

nodes = pipeline.run(documents=documents, show_progress=True);

/Users/omar/Documents/ai_repos/ai-tutor-rag-system/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 108/108 [00:02<00:00, 45.21it/s]


In [12]:
len( nodes )

108

# Load Indexes

In [13]:
# Create your index
db = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = db.get_or_create_collection("mini-llama-articles")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [14]:
# Create your index
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(vector_store)

# Query Dataset

In [15]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
query_engine = index.as_query_engine()

res = query_engine.query("How many parameters LLaMA2 model has?")

In [16]:
res.response

'The Llama 2 model is available in four different sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.'

In [17]:
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("Text\t", src.text)
  print("Score\t", src.score)
  print("-_"*20)

Node ID	 f707756065d1f788b41fb97fcef81979e1fd241dbfa4034a24bec8e57b648482
Title	 Meta's Llama 2: Revolutionizing Open Source Language Models for Commercial Use
Text	 I. Llama 2: Revolutionizing Commercial Use Unlike its predecessor Llama 1, which was limited to research use, Llama 2 represents a major advancement as an open-source commercial model. Businesses can now integrate Llama 2 into products to create AI-powered applications. Availability on Azure and AWS facilitates fine-tuning and adoption. However, restrictions apply to prevent exploitation. Companies with over 700 million active daily users cannot use Llama 2. Additionally, its output cannot be used to improve other language models.  II. Llama 2 Model Flavors Llama 2 is available in four different model sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters. While 7B, 13B, and 70B have already been released, the 34B model is still awaited. The pretrained variant, trained on a whopping 2 trillion tokens, boasts a

### Trying a different Query

In [18]:
res = query_engine.query("Can LLaMA2 do NLU?")

In [19]:
res.response

'Yes, LLaMA2 can do NLU (Natural Language Understanding) as mentioned in the context information.'

In [20]:
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("Text\t", src.text)
  print("Score\t", src.score)
  print("-_"*20)

Node ID	 8cf94b9369ba8da18d02172b9cbf885afb60cddd0a2381a86a81ca8e6a9b10f9
Title	 Exploring Large Language Models -Part 3
Text	 LM model training via UnSupervised learning). Note that this model was loaded in 4-bit, making it runnable on a single T4 GPU and trained with QLoRa. With QLoRA, only a fraction of the adapter weights are trained and summed with the existing frozen pre-trained weights of the model during inference. Here is an illustrative Colab notebook. You can see that training the model with just the text as is, does not result in proper output to questions. The answers are not affected by the training data. Take 2: Instruct Fine-tuning with QLoRa Instruction Tuning concept is a higher-level training concept introduced by this paper FineTuned Language Models Are Zero shot Learners (FLAN) We leverage the intuition that NLP tasks can be described via natural language instructions, such as "Is the sentiment of this movie review positive or negative?" or "Translate 'how are you'

From the articles:
  
> [...]The 7 billion model of Llama2 has sufficient NLU (Natural Language Understanding) to create output based on a particular format[...]



# No Metadata

In [21]:
documents_no_meta = [Document(text=row[1]) for row in rows]

# By default, the node/chunks ids are set to random uuids. To ensure same id's per run, we manually set them.
for idx, doc in enumerate(documents_no_meta):
    doc.id_ = f"doc_{idx}"

In [22]:
from llama_index.embeddings import OpenAIEmbedding
from llama_index.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        OpenAIEmbedding(),
    ]
)

nodes_no_meta = pipeline.run(documents=documents_no_meta, show_progress=True)

Generating embeddings: 100%|██████████| 94/94 [00:01<00:00, 67.81it/s]


In [23]:
from llama_index import ServiceContext

index_no_metadata = VectorStoreIndex(
    nodes=nodes_no_meta,
    service_context=ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo")),
)

In [24]:
query_engine_no_metadata = index_no_metadata.as_query_engine()

In [25]:
res = query_engine_no_metadata.query("Can LLaMA2 do NLU?")

In [26]:
res.response

'LLaMA2 is a foundational language model developed by Meta. While the context does not explicitly mention whether LLaMA2 can do Natural Language Understanding (NLU), it is described as a research tool for researchers and academics to understand and study NLP models. Therefore, it is possible that LLaMA2 has capabilities related to NLU, but without further information, it cannot be definitively stated.'

In [27]:
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Text\t", src.text)
  print("Score\t", src.score)
  print("-_"*20)

Node ID	 e97bbe3d37bacb34902b4db67351799f1309541d4879e53b97fad08a4417304f
Text	 LLaMA: Meta's new AI tool According to the official release, LLaMA is a foundational language model developed to assist 'researchers and academics' in their work (as opposed to the average web user) to understand and study these NLP models. Leveraging AI in such a way could give researchers an edge in terms of time spent. You may not know this, but this would be Meta's third LLM after Blender Bot 3 and Galactica. However, the two LLMs were shut down soon, and Meta stopped their further development, as it produced erroneous results. Before moving further, it is important to emphasize that LLaMA is NOT a chatbot like ChatGPT. As I mentioned before, it is a 'research tool' for researchers. We can expect the initial versions of LLaMA to be a bit more technical and indirect to use as opposed to the case with ChatGPT, which was very direct, interactive, and a lot easy to use. "Smaller, more performant models such

# Evaluate

In [28]:
from llama_index.evaluation import generate_question_context_pairs
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
rag_eval_dataset = generate_question_context_pairs(
    nodes,
    llm=llm,
    num_questions_per_chunk=1
)

# We can save the dataset as a json file for later use.
rag_eval_dataset.save_json("./rag_eval_dataset.json")

100%|██████████| 108/108 [05:05<00:00,  2.83s/it]


In [29]:
from llama_index.finetuning.embeddings.common import (
    EmbeddingQAFinetuneDataset,
)
rag_eval_dataset = EmbeddingQAFinetuneDataset.from_json(
    "./rag_eval_dataset.json"
)

In [30]:
import pandas as pd

def display_results_retriever(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

In [31]:
from llama_index.evaluation import RetrieverEvaluator

# We can evaluate the retievers with different top_k values.
for i in [2, 4, 6, 8, 10]:
    retriever = index.as_retriever(similarity_top_k=i)
    retriever_evaluator = RetrieverEvaluator.from_metric_names(
        ["mrr", "hit_rate"], retriever=retriever
    )
    eval_results = await retriever_evaluator.aevaluate_dataset(rag_eval_dataset)
    print(display_results_retriever(f"Retriever top_{i}", eval_results))

    Retriever Name  Hit Rate       MRR
0  Retriever top_2  0.638646  0.523472
    Retriever Name  Hit Rate       MRR
0  Retriever top_4  0.776201  0.564683
    Retriever Name  Hit Rate       MRR
0  Retriever top_6  0.819869  0.572726
    Retriever Name  Hit Rate       MRR
0  Retriever top_8  0.840611  0.575494
     Retriever Name  Hit Rate       MRR
0  Retriever top_10  0.854803  0.576937


In [32]:
from llama_index.evaluation import RelevancyEvaluator, FaithfulnessEvaluator, BatchEvalRunner
from llama_index import ServiceContext
from llama_index.llms import OpenAI

for i in [2, 4, 6, 8, 10]:
    # Set Faithfulness and Relevancy evaluators
    query_engine = index.as_query_engine(similarity_top_k=i)

    # While we use GPT3.5-Turbo to answer questions, we can use GPT4 to evaluate the answers.
    llm_gpt4 = OpenAI(temperature=0, model="gpt-4-1106-preview")
    service_context_gpt4 = ServiceContext.from_defaults(llm=llm_gpt4)

    faithfulness_evaluator = FaithfulnessEvaluator(service_context=service_context_gpt4)
    relevancy_evaluator = RelevancyEvaluator(service_context=service_context_gpt4)

    # Run evaluation
    queries = list(rag_eval_dataset.queries.values())
    batch_eval_queries = queries[:20]

    runner = BatchEvalRunner(
    {"faithfulness": faithfulness_evaluator, "relevancy": relevancy_evaluator},
    workers=8,
    )
    eval_results = await runner.aevaluate_queries(
        query_engine, queries=batch_eval_queries
    )
    faithfulness_score = sum(result.passing for result in eval_results['faithfulness']) / len(eval_results['faithfulness'])
    print(f"top_{i} faithfulness_score: {faithfulness_score}")

    relevancy_score = sum(result.passing for result in eval_results['faithfulness']) / len(eval_results['relevancy'])
    print(f"top_{i} relevancy_score: {relevancy_score}")
    print("-_"*10)

top_2 faithfulness_score: 0.95
top_2 relevancy_score: 0.95
-_-_-_-_-_-_-_-_-_-_
top_4 faithfulness_score: 1.0
top_4 relevancy_score: 1.0
-_-_-_-_-_-_-_-_-_-_
top_6 faithfulness_score: 1.0
top_6 relevancy_score: 1.0
-_-_-_-_-_-_-_-_-_-_
top_8 faithfulness_score: 1.0
top_8 relevancy_score: 1.0
-_-_-_-_-_-_-_-_-_-_
top_10 faithfulness_score: 1.0
top_10 relevancy_score: 1.0
-_-_-_-_-_-_-_-_-_-_


# Evaluate No Metadata

In [33]:
from llama_index.evaluation import RetrieverEvaluator

# We can evaluate the retievers with different top_k values.
for i in [2, 4, 6, 8, 10]:
    retriever = index_no_metadata.as_retriever(similarity_top_k=i)
    retriever_evaluator = RetrieverEvaluator.from_metric_names(
        ["mrr", "hit_rate"], retriever=retriever
    )
    eval_results = await retriever_evaluator.aevaluate_dataset(rag_eval_dataset)
    print(display_results_retriever(f"Retriever top_{i}", eval_results))

    Retriever Name  Hit Rate       MRR
0  Retriever top_2  0.394105  0.324236
    Retriever Name  Hit Rate       MRR
0  Retriever top_4  0.529476  0.364447
    Retriever Name  Hit Rate       MRR
0  Retriever top_6  0.575328  0.372889
    Retriever Name  Hit Rate       MRR
0  Retriever top_8  0.605895  0.376944
     Retriever Name  Hit Rate       MRR
0  Retriever top_10  0.624454  0.378897


In [34]:
from llama_index.evaluation import RelevancyEvaluator, FaithfulnessEvaluator, BatchEvalRunner
from llama_index import ServiceContext
from llama_index.llms import OpenAI

for i in [2, 4, 6, 8, 10]:
    # Set Faithfulness and Relevancy evaluators
    query_engine = index_no_metadata.as_query_engine(similarity_top_k=i)

    # While we use GPT3.5-Turbo to answer questions, we can use GPT4 to evaluate the answers.
    llm_gpt4 = OpenAI(temperature=0, model="gpt-4-1106-preview")
    service_context_gpt4 = ServiceContext.from_defaults(llm=llm_gpt4)

    faithfulness_evaluator = FaithfulnessEvaluator(service_context=service_context_gpt4)
    relevancy_evaluator = RelevancyEvaluator(service_context=service_context_gpt4)

    # Run evaluation
    queries = list(rag_eval_dataset.queries.values())
    batch_eval_queries = queries[:20]

    runner = BatchEvalRunner(
    {"faithfulness": faithfulness_evaluator, "relevancy": relevancy_evaluator},
    workers=8,
    )
    eval_results = await runner.aevaluate_queries(
        query_engine, queries=batch_eval_queries
    )
    faithfulness_score = sum(result.passing for result in eval_results['faithfulness']) / len(eval_results['faithfulness'])
    print(f"top_{i} faithfulness_score: {faithfulness_score}")

    relevancy_score = sum(result.passing for result in eval_results['faithfulness']) / len(eval_results['relevancy'])
    print(f"top_{i} relevancy_score: {relevancy_score}")
    print("-_"*10)

top_2 faithfulness_score: 1.0
top_2 relevancy_score: 1.0
-_-_-_-_-_-_-_-_-_-_
top_4 faithfulness_score: 1.0
top_4 relevancy_score: 1.0
-_-_-_-_-_-_-_-_-_-_
top_6 faithfulness_score: 1.0
top_6 relevancy_score: 1.0
-_-_-_-_-_-_-_-_-_-_
top_8 faithfulness_score: 1.0
top_8 relevancy_score: 1.0
-_-_-_-_-_-_-_-_-_-_
top_10 faithfulness_score: 0.95
top_10 relevancy_score: 0.95
-_-_-_-_-_-_-_-_-_-_
